## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,MG,-20.2833,44.2833,63.68,78,0,4.97,clear sky
1,1,Cape Town,KI,3.0707,172.7902,81.84,78,11,12.53,light rain
2,2,Nambucca Heads,ZA,-33.9258,18.4232,59.38,77,0,3.78,clear sky
3,3,Awbari,AU,-30.6500,153.0000,68.16,51,0,6.93,clear sky
4,4,Solnechnyy,LY,26.5921,12.7805,77.18,16,0,6.29,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for this trip? "))
max_temp = float(input("What is the maximum temperature you would like for this trip? "))

What is the minimum temperature you would like for this trip? 70
What is the maximum temperature you would like for this trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
vacation_cities_df=city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
vacation_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Cape Town,KI,3.0707,172.7902,81.84,78,11,12.53,light rain
4,4,Solnechnyy,LY,26.5921,12.7805,77.18,16,0,6.29,clear sky
5,5,Mataura,RU,50.7214,136.6319,81.05,57,100,5.84,overcast clouds
7,7,Castro,CV,14.8961,-24.4956,78.26,88,92,3.98,overcast clouds
9,9,Asfi,PF,-16.7000,-151.0167,77.97,77,12,26.02,light rain


In [5]:
# 4a. Determine if there are any empty rows.
vacation_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
vacation_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Cape Town,KI,3.0707,172.7902,81.84,78,11,12.53,light rain
4,4,Solnechnyy,LY,26.5921,12.7805,77.18,16,0,6.29,clear sky
5,5,Mataura,RU,50.7214,136.6319,81.05,57,100,5.84,overcast clouds
7,7,Castro,CV,14.8961,-24.4956,78.26,88,92,3.98,overcast clouds
9,9,Asfi,PF,-16.7000,-151.0167,77.97,77,12,26.02,light rain
...,...,...,...,...,...,...,...,...,...,...
720,720,Surt,PE,-3.7481,-73.2472,73.44,100,75,3.44,broken clouds
721,721,Axim,LY,31.2089,16.5887,79.02,60,0,3.38,clear sky
722,722,Greenville,GH,4.8699,-2.2405,75.42,82,96,13.27,overcast clouds
723,723,Campo Maior,US,35.6127,-77.3663,80.76,92,0,5.75,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = vacation_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Cape Town,KI,81.84,light rain,3.0707,172.7902,
4,Solnechnyy,LY,77.18,clear sky,26.5921,12.7805,
5,Mataura,RU,81.05,overcast clouds,50.7214,136.6319,
7,Castro,CV,78.26,overcast clouds,14.8961,-24.4956,
9,Asfi,PF,77.97,light rain,-16.7000,-151.0167,
19,Nador,PF,74.03,overcast clouds,-23.1203,-134.9692,
20,Kalininsk,MA,71.85,clear sky,35.1740,-2.9287,
21,Taolanaro,RU,72.07,overcast clouds,51.5000,44.4758,
27,Victoria,PF,77.63,light rain,-9.8000,-139.0333,
31,Scottsbluff,OM,82.13,overcast clouds,22.5667,59.5289,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
        
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Cape Town,KI,81.84,light rain,3.0707,172.7902,Isles Sunset Lodge
4,Solnechnyy,LY,77.18,clear sky,26.5921,12.7805,فندق أوباري
5,Mataura,RU,81.05,overcast clouds,50.7214,136.6319,Hotel complex Pastoral
7,Castro,CV,78.26,overcast clouds,14.8961,-24.4956,Hotel Xaguate
9,Asfi,PF,77.97,light rain,-16.7000,-151.0167,Maitai Lapita Village Huahine
19,Nador,PF,74.03,overcast clouds,-23.1203,-134.9692,People ThankYou
20,Kalininsk,MA,71.85,clear sky,35.1740,-2.9287,Hôtel La Giralda
21,Taolanaro,RU,72.07,overcast clouds,51.5000,44.4758,Guest House on Chekhova
27,Victoria,PF,77.63,light rain,-9.8000,-139.0333,Villa Enata
31,Scottsbluff,OM,82.13,overcast clouds,22.5667,59.5289,Sur Plaza Hotel


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Cape Town,KI,81.84,light rain,3.0707,172.7902,Isles Sunset Lodge
4,Solnechnyy,LY,77.18,clear sky,26.5921,12.7805,فندق أوباري
5,Mataura,RU,81.05,overcast clouds,50.7214,136.6319,Hotel complex Pastoral
7,Castro,CV,78.26,overcast clouds,14.8961,-24.4956,Hotel Xaguate
9,Asfi,PF,77.97,light rain,-16.7000,-151.0167,Maitai Lapita Village Huahine
...,...,...,...,...,...,...,...
720,Surt,PE,73.44,broken clouds,-3.7481,-73.2472,Cortez Malaga Techy Caroly
721,Axim,LY,79.02,clear sky,31.2089,16.5887,الهادي ابوشعاله سرت
722,Greenville,GH,75.42,overcast clouds,4.8699,-2.2405,Axim Beach Resort And Spa
723,Campo Maior,US,80.76,clear sky,35.6127,-77.3663,"Holiday Inn Express Greenville, an IHG Hotel"


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 

max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))